# Terrier-Spark Example Notebook

More example notebooks can be found at https://github.com/terrier-org/terrier-spark/tree/master/example_notebooks/toree

Firstly, we need to download the dependencies.

In [1]:
//This can take a minute or two.
%AddDeps org.terrier terrier-core 5.0 --transitive --repository file:/root/.m2/repository --exclude org.slf4j:slf4j-log4j12  
%AddDeps org.terrier terrier-spark 0.0.1-SNAPSHOT --repository file:/root/.m2/repository --transitive

Marking org.terrier:terrier-core:5.0 for download
Obtained 295 files
Marking org.terrier:terrier-spark:0.0.1-SNAPSHOT for download
Obtained 336 files


In [3]:
//Lets check we have Terrier downloaded.
org.terrier.Version.VERSION

5.0

This cell does a TREC run

In [ ]:
val indexref = IndexRef.of("/path/to/index/data.properties")

val props = Map(
"terrier.home" -> terrierHome)

TopicSource.configureTerrier(props)
val topics = TopicSource.extractTRECTopics(topicsFile)
    .toList.toDF("qid", "query")

val queryTransform = new QueryingTransformer()
    .setTerrierProperties(props)
    .setIndexReference(indexref)
    .setSampleModel(model)

val r1 = queryTransform.transform(topics)
//r1 is a dataframe with results for queries in topics
val qrelTransform = new QrelTransformer()
    .setQrelsFile(qrelsFile)

val r2 = qrelTransform.transform(r1)
//r2 is a dataframe as r1, but also includes a label column
val ndcg = new RankingEvaluator(Measure.NDCG, 20).evaluateByQuery(r2).toList

ndcg